# **PROJETO CHICO**
por **Rodrigo Gosling**
<br>
<a href="https://www.linkedin.com/in/rodgosling/" target="_blank">Linkedin</a>
<br>
---

O Projeto Chico foi elaborado durante a 2ª Imersão de dados da <a href="https://www.alura.com.br/" target="_blank">Alura</a>. Neste estudo foram elaborados dois notebooks: 
* "Obtendo o datasete ENEM 2019 MG" (este), onde foram realizados os o processos de obtenção, tratamento, limpeza da base de dados do ENEM 2019 filtrados pelo estado de Minas Gerais.
* "Análises", onde foram realizadas as análises do dataset.
<br>
---

# Obtenção das bases de dados

In [1]:
# Importando bibliotecas necessárias

import numpy          as np
import pandas         as pd
import dask.dataframe as dd

## Dados da população estimada 2020

O conjunto de dados referente a quantidade de habitantes de cada município do estado de Minas Gerais será utilizado para classificar as cidades quanto ao seu tamanho, conforme quadro abaixo:

Tamanho | População
--- | ---
Pequeno | até 50.000 hab.
Médio | entre 50.0001 e 300.000 hab.
Grande | acima de 300.001 hab.

<br>
Fonte: <a href="https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?edicao=28674&t=resultados">IBGE 2020</a>

In [2]:
# Obtendo tabela com população estimada por município IBGE2020
# Fonte: IBGE 2020
pop_est = pd.read_excel('estimativa_dou_2020.xls', sheet_name=1, header = 1)


# Obtendo população estimada do estado Minas Gerais
pop_est1 = pop_est[pop_est['UF'] == 'MG']
pop_est1.reset_index(inplace = True)
pop_est_mg = pop_est1.copy()
pop_est_mg.drop(columns = ['COD. MUNIC', 'COD. UF','index'], inplace = True)

In [3]:
# Classificando as cidades de acordo com o número de habitantes

# Cidades com população até 20.000 hab. -- Pequena
# Cidades com população entre 20.001 e 500.000 hab. -- Média
# Cidades com população acima de 500.000 hab. -- Grande
# Capital do estado -- Belo Horizonte

uf_capital = 'Belo Horizonte'

pop_est_mg['TAMANHO'] = np.where(pop_est_mg['POPULAÇÃO ESTIMADA'] >= 300001, 'Grande',
                                 np.where(pop_est_mg['POPULAÇÃO ESTIMADA'] <= 50000, 'Pequeno',
                                          np.where(pop_est_mg['POPULAÇÃO ESTIMADA'].between(50001,300000), 'Médio', 'Erro')))
                            

pop_est_mg.loc[pop_est_mg['NOME DO MUNICÍPIO'] == uf_capital,'TAMANHO'] = 'Capital'
                                                       

### Exportando o dataset

In [4]:
# Exportando dataset para arquivo .csv
pop_est_mg.to_csv('POP_ESTIMADA_MG.csv')

<hr>

<br>

## Dataset ENEM 2019

O conjunto de dados do ENEM 2019 é o objeto de estudos deste projeto e foi obtido através do site do INEP. 

Devido a limitações técnicas, este projeto se limita a estudar os dados do estado de Minas Gerais.
 
<br>
Fonte: <a href="http://inep.gov.br/microdados">INEP 2019</a>

In [ ]:
# Obtendo dataset ENEM 2019
# Fonte: INEP 2019 

#  Importando o dataset completo
df_raw = dd.read_csv('MICRODADOS_ENEM_2019.csv', encoding = 'ISO-8859-1', sep = ';')

# Filtrando dados da UF MG
df_raw = df_raw[df_raw.SG_UF_RESIDENCIA == 'MG']

# Importando dataset para arquivo .csv
df_raw.to_csv('ENEM_2019_MG.csv')

### Dataset ENEM 2019 - Minas Gerais

In [5]:
# Obtendo base de dados ENEM 2019 MG -- Selecionando somente as colunas necessárias para as análises

columns_to_keep = ['NO_MUNICIPIO_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'IN_TREINEIRO',
                                'NO_MUNICIPIO_ESC', 'TP_LOCALIZACAO_ESC', 'NO_MUNICIPIO_PROVA', 'SG_UF_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
                                 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006',
                                 'Q007', 'Q008', 'Q010', 'Q011', 'Q022', 'Q024', 'Q025']

enem_mg = pd.read_csv("ENEM_2019_MG.txt", sep=";", usecols = columns_to_keep)


# Criando um df com cópia da base de dados MG
dados_mg = enem_mg.copy()


# Removendo a coluna SG_UF_RESIDENCIA que não é mais relevante para a análise
dados_mg.drop(columns = ['SG_UF_RESIDENCIA'], inplace = True)


# Criando coluna TAM_MUNICÍPIO baseada na tabela de dados da populaçõa estimada 2020
dados_mg['TAM_MUNICIPIO'] = 'NaN'

chico_mg = dados_mg.copy()

for i in range(len(dados_mg)):
    
    nome_mun_loop = dados_mg.iloc[i]['NO_MUNICIPIO_RESIDENCIA']
    cond_mun_loop = pop_est_mg.loc[pop_est_mg['NOME DO MUNICÍPIO'] == nome_mun_loop]['NOME DO MUNICÍPIO'] == nome_mun_loop
    pop_mun_loop  = pop_est_mg.loc[pop_est_mg['NOME DO MUNICÍPIO'] == nome_mun_loop]['TAMANHO']
    
    chico_mg.TAM_MUNICIPIO.iloc[i] = np.where(cond_mun_loop, pop_mun_loop, 'Erro')

        
# Criando coluna NOTA TOTAL, somando as notas de todas as provas
provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
chico_mg['NU_NOTA_TOTAL'] = chico_mg[provas].sum(axis = 1)

 

C:\Users\rodgo\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


### Limpeza e correção de erros

In [6]:
# Limpando resíduos da coluna TAM_MUNICIPIO e criando a coluna Tipo Município
chico_mg['TP_MUNICIPIO'] = np.where(chico_mg['TAM_MUNICIPIO'] == 'Grande', 'Grande', 
                             np.where(chico_mg['TAM_MUNICIPIO'] == 'Médio', 'Médio',
                                np.where(chico_mg['TAM_MUNICIPIO'] == 'Pequeno', 'Pequeno',
                                   np.where(chico_mg['TAM_MUNICIPIO'] == 'Capital', 'Capital', 'Erro'))))



# Correção das cidades com divergência no nome e foram classificadas como 'Erro'
# São Thomé das Letras    7120 -- Pequena
# Dona Eusébia            6619 -- Pequena
# Pingo d'Água            4986 -- Pequena


chico_mg.TP_MUNICIPIO.replace('Erro','Pequeno', inplace = True)


# Deletando a coluna TAM_MUNICIPIO que tornou-se desnecessária
chico_mg.drop(columns = 'TAM_MUNICIPIO', inplace = True)

chico_mg.head()

,NO_MUNICIPIO_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ESCOLA,IN_TREINEIRO,NO_MUNICIPIO_ESC,TP_LOCALIZACAO_ESC,...,Q006,Q007,Q008,Q010,Q011,Q022,Q024,Q025,NU_NOTA_TOTAL,TP_MUNICIPIO
0,Uberlândia,21,M,1,1,1,1,0,NaN,NaN,...,I,A,C,B,A,D,B,B,3097.0,Grande
1,Canaã,20,M,1,1,1,1,0,NaN,NaN,...,D,A,B,A,B,C,A,A,0.0,Pequeno
2,Araxá,24,M,1,2,1,1,0,NaN,NaN,...,F,A,B,A,A,C,A,B,0.0,Médio
3,Belo Horizonte,33,M,1,1,1,1,0,NaN,NaN,...,P,B,D,C,A,D,D,B,0.0,Capital
4,São João da Ponte,23,M,1,3,1,1,0,NaN,NaN,...,D,A,B,B,B,C,B,A,3398.0,Pequeno


### Exportando o dataset

In [13]:
# Exportando dataset para arquivo .csv
chico_mg.to_csv('ENEM_2019_MG_CHICO.csv')

<hr>